# ¿Cuál es la mejor tarifa?

Trabajamos como analista para el operador de telecomunicaciones Megaline. La empresa ofrece a sus clientes dos tarifas de prepago, Surf y Ultimate. El departamento comercial quiere saber cuál de las tarifas genera más ingresos para poder ajustar el presupuesto de publicidad.

Vas a realizar un análisis preliminar de las tarifas basado en una selección de clientes relativamente pequeña. Tendrás los datos de 500 clientes de Megaline: quiénes son los clientes, de dónde son, qué tarifa usan, así como la cantidad de llamadas que hicieron y los mensajes de texto que enviaron en 2018. Tu trabajo es analizar el comportamiento de los clientes y determinar qué tarifa de prepago genera más ingresos.

## Inicialización

In [1]:
# Cargar todas las librerías
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats as st

## Cargar datos

In [2]:
# Carga los archivos de datos en diferentes DataFrames
df_calls=pd.read_csv("/datasets/megaline_calls.csv")
df_internet=pd.read_csv("/datasets/megaline_internet.csv")
df_messages=pd.read_csv("/datasets/megaline_messages.csv")
df_plans=pd.read_csv("/datasets/megaline_plans.csv")
df_users=pd.read_csv("/datasets/megaline_users.csv")

## Preparar los datos

## Tarifas

In [3]:
# Imprime la información general/resumida sobre el DataFrame de las tarifas
df_plans.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   messages_included      2 non-null      int64  
 1   mb_per_month_included  2 non-null      int64  
 2   minutes_included       2 non-null      int64  
 3   usd_monthly_pay        2 non-null      int64  
 4   usd_per_gb             2 non-null      int64  
 5   usd_per_message        2 non-null      float64
 6   usd_per_minute         2 non-null      float64
 7   plan_name              2 non-null      object 
dtypes: float64(2), int64(5), object(1)
memory usage: 256.0+ bytes


In [4]:
# Imprime una muestra de los datos para las tarifas
df_plans.head()


,messages_included,mb_per_month_included,minutes_included,usd_monthly_pay,usd_per_gb,usd_per_message,usd_per_minute,plan_name
0,50,15360,500,20,10,0.03,0.03,surf
1,1000,30720,3000,70,7,0.01,0.01,ultimate




No veo ningun problema, los tipos de datos coinciden bien, no hay datos ausentes tampoco, no es necesario hacer cambios generales.



## Usuarios/as

In [5]:
# Imprime la información general/resumida sobre el DataFrame de usuarios
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     500 non-null    int64 
 1   first_name  500 non-null    object
 2   last_name   500 non-null    object
 3   age         500 non-null    int64 
 4   city        500 non-null    object
 5   reg_date    500 non-null    object
 6   plan        500 non-null    object
 7   churn_date  34 non-null     object
dtypes: int64(2), object(6)
memory usage: 31.4+ KB


In [6]:
# Imprime una muestra de datos para usuarios
df_users.head()


,user_id,first_name,last_name,age,city,reg_date,plan,churn_date
0,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaN
1,1001,Mickey,Wilkerson,28,"Seattle-Tacoma-Bellevue, WA MSA",2018-08-13,surf,NaN
2,1002,Carlee,Hoffman,36,"Las Vegas-Henderson-Paradise, NV MSA",2018-10-21,surf,NaN
3,1003,Reynaldo,Jenkins,52,"Tulsa, OK MSA",2018-01-28,surf,NaN
4,1004,Leonila,Thompson,40,"Seattle-Tacoma-Bellevue, WA MSA",2018-05-23,surf,NaN



No hay nada que corregir, lo que podemos hacer es que la columna "churn_date" remplazaremos los datos ausentes por un String que sera "using"

### Corregir los datos

In [7]:
#Transformamos las fechas
df_users["reg_date"]= pd.to_datetime(df_users["reg_date"], format="%Y-%m-%d")
df_users["churn_date"] = pd.to_datetime(df_users["churn_date"], format="%Y-%m-%d")

### Enriquecer los datos

In [8]:
#No eliminaremos los datos ausentes ya que le quitaria la propiedad de datetime a la columna churn_date
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     500 non-null    int64         
 1   first_name  500 non-null    object        
 2   last_name   500 non-null    object        
 3   age         500 non-null    int64         
 4   city        500 non-null    object        
 5   reg_date    500 non-null    datetime64[ns]
 6   plan        500 non-null    object        
 7   churn_date  34 non-null     datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(4)
memory usage: 31.4+ KB


## Llamadas

In [9]:
# Imprime la información general/resumida sobre el DataFrame de las llamadas
df_calls.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137735 entries, 0 to 137734
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id         137735 non-null  object 
 1   user_id    137735 non-null  int64  
 2   call_date  137735 non-null  object 
 3   duration   137735 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 4.2+ MB


In [10]:
# Imprime una muestra de datos para las llamadas
df_calls.head()

,id,user_id,call_date,duration
0,1000_93,1000,2018-12-27,8.52
1,1000_145,1000,2018-12-27,13.66
2,1000_247,1000,2018-12-27,14.48
3,1000_309,1000,2018-12-28,5.76
4,1000_380,1000,2018-12-30,4.22



No es necesario

### Corregir los datos

In [11]:
#Transformamos las fechas
df_calls["call_date"]= pd.to_datetime(df_calls["call_date"], format="%Y-%m-%d")

## Mensajes

In [12]:
# Imprime la información general/resumida sobre el DataFrame de los mensajes
df_messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76051 entries, 0 to 76050
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            76051 non-null  object
 1   user_id       76051 non-null  int64 
 2   message_date  76051 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.7+ MB


In [13]:
# Imprime una muestra de datos para los mensajes
df_messages.head()


,id,user_id,message_date
0,1000_125,1000,2018-12-27
1,1000_160,1000,2018-12-31
2,1000_223,1000,2018-12-31
3,1000_251,1000,2018-12-27
4,1000_255,1000,2018-12-26


No es necesario corregir

### Corregir los datos

In [14]:
#Transformamos las fechas
df_messages["message_date"]= pd.to_datetime(df_messages["message_date"], format="%Y-%m-%d")

## Internet

In [15]:
# Imprime la información general/resumida sobre el DataFrame de internet
df_internet.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104825 entries, 0 to 104824
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            104825 non-null  object 
 1   user_id       104825 non-null  int64  
 2   session_date  104825 non-null  object 
 3   mb_used       104825 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 3.2+ MB


In [16]:
# Imprime una muestra de datos para el tráfico de internet
df_internet.head()


,id,user_id,session_date,mb_used
0,1000_13,1000,2018-12-29,89.86
1,1000_204,1000,2018-12-31,0.00
2,1000_379,1000,2018-12-28,660.40
3,1000_413,1000,2018-12-26,270.99
4,1000_442,1000,2018-12-27,880.22


Podemos notar que hay un mensaje que no uso MB, pero fuera de esto todo esta bien.

### Corregir los datos

In [17]:
#Transformamos las fechas
df_internet["session_date"]= pd.to_datetime(df_internet["session_date"], format="%Y-%m-%d")

## Estudiar las condiciones de las tarifas

In [18]:
# Imprime las condiciones de la tarifa y asegúrate de que te quedan claras
df_plans.head()

,messages_included,mb_per_month_included,minutes_included,usd_monthly_pay,usd_per_gb,usd_per_message,usd_per_minute,plan_name
0,50,15360,500,20,10,0.03,0.03,surf
1,1000,30720,3000,70,7,0.01,0.01,ultimate


## Agregar datos por usuario


In [19]:
# Calcula el número de llamadas hechas por cada usuario al mes. Guarda el resultado.
calls_per_user=df_calls.pivot_table(index='user_id',columns=df_calls["call_date"].dt.month,values="duration",aggfunc='count')
display(calls_per_user)
#Eliminamos datos nulos
calls_per_user=calls_per_user.fillna(0)
calls_per_user.info()
#Cambiamos los nombres numericos de los meses a sus respectivos nombres en ingles
calls_per_user = calls_per_user.rename(columns={1:'January',2:'February',3:'March',4:'April',5:'May',6:'June',7:'July',8:'August',9:'September',10:'October',11:'November',12:'December'})
display(calls_per_user)

call_date,1,2,3,4,5,6,7,8,9,10,11,12
user_id,,,,,,,,,,,,
1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0
1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.0,49.0,65.0,64.0,56.0
1002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,55.0,47.0
1003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,149.0
1004,NaN,NaN,NaN,NaN,21.0,44.0,49.0,49.0,42.0,61.0,54.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.0,75.0,61.0,65.0
1496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,52.0,46.0,39.0,40.0
1497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 481 entries, 1000 to 1499
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1       481 non-null    float64
 1   2       481 non-null    float64
 2   3       481 non-null    float64
 3   4       481 non-null    float64
 4   5       481 non-null    float64
 5   6       481 non-null    float64
 6   7       481 non-null    float64
 7   8       481 non-null    float64
 8   9       481 non-null    float64
 9   10      481 non-null    float64
 10  11      481 non-null    float64
 11  12      481 non-null    float64
dtypes: float64(12)
memory usage: 48.9 KB


call_date,January,February,March,April,May,June,July,August,September,October,November,December
user_id,,,,,,,,,,,,
1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0
1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0,49.0,65.0,64.0,56.0
1002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,55.0,47.0
1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,149.0
1004,0.0,0.0,0.0,0.0,21.0,44.0,49.0,49.0,42.0,61.0,54.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52.0,75.0,61.0,65.0
1496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,52.0,46.0,39.0,40.0
1497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54.0


In [20]:
# Calcula el número de mensajes enviados por cada usuario al mes. Guarda el resultado.
#Realizamos el mismo proceso
message_per_user=df_messages.pivot_table(index='user_id',columns=df_messages["message_date"].dt.month,values="id",aggfunc='count')
message_per_user=message_per_user.fillna(0)
message_per_user = message_per_user.rename(columns={1:'January',2:'February',3:'March',4:'April',5:'May',6:'June',7:'July',8:'August',9:'September',10:'October',11:'November',12:'December'})
display(message_per_user)

message_date,January,February,March,April,May,June,July,August,September,October,November,December
user_id,,,,,,,,,,,,
1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0
1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,44.0,53.0,36.0,44.0
1002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,32.0,41.0
1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0
1004,0.0,0.0,0.0,0.0,7.0,18.0,26.0,25.0,21.0,24.0,25.0,31.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1491,0.0,0.0,0.0,6.0,45.0,54.0,64.0,50.0,50.0,51.0,46.0,43.0
1492,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,29.0,31.0,37.0
1494,0.0,0.0,0.0,0.0,0.0,0.0,20.0,27.0,21.0,38.0,35.0,33.0


In [21]:
# Calcula la cantidad de minutos usados por cada usuario al mes. Guarda el resultado.
minutes_per_user=df_calls.pivot_table(index='user_id',columns=df_calls["call_date"].dt.month,values="duration",aggfunc='sum')
minutes_per_user=minutes_per_user.fillna(0)
minutes_per_user = minutes_per_user.rename(columns={1:'January',2:'February',3:'March',4:'April',5:'May',6:'June',7:'July',8:'August',9:'September',10:'October',11:'November',12:'December'})
display(minutes_per_user)

call_date,January,February,March,April,May,June,July,August,September,October,November,December
user_id,,,,,,,,,,,,
1000,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,116.83
1001,0.0,0.00,0.00,0.00,0.00,0.00,0.00,171.14,297.69,374.11,404.59,392.93
1002,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,54.13,359.76,363.24
1003,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1041.00
1004,0.0,0.00,0.00,0.00,181.58,261.32,358.45,334.86,284.60,341.63,452.98,403.53
...,...,...,...,...,...,...,...,...,...,...,...,...
1495,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,283.46,578.03,337.45,467.47
1496,0.0,0.00,0.00,0.00,0.00,0.00,0.00,114.62,389.94,301.16,291.88,278.61
1497,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,276.53


In [22]:
# Calcula el volumen del tráfico de Internet usado por cada usuario al mes. Guarda el resultado.
mb_used_per_user=df_internet.pivot_table(index='user_id',columns=df_internet["session_date"].dt.month,values="mb_used",aggfunc='sum')
mb_used_per_user=mb_used_per_user.fillna(0)
mb_used_per_user = mb_used_per_user.rename(columns={1:'January',2:'February',3:'March',4:'April',5:'May',6:'June',7:'July',8:'August',9:'September',10:'October',11:'November',12:'December'})
display(mb_used_per_user)

session_date,January,February,March,April,May,June,July,August,September,October,November,December
user_id,,,,,,,,,,,,
1000,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1901.47
1001,0.0,0.00,0.00,0.00,0.00,0.00,0.00,6919.15,13314.82,22330.49,18504.30,19369.18
1002,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6552.01,19345.08,14396.24
1003,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,27044.14
1004,0.0,0.00,0.00,0.00,6547.21,20672.82,24516.62,27981.74,18852.72,14541.63,21850.78,21389.29
...,...,...,...,...,...,...,...,...,...,...,...,...
1495,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,22981.37,26899.41,24912.78,24097.40
1496,0.0,0.00,0.00,0.00,0.00,0.00,0.00,8605.66,16389.27,14287.36,8547.36,16438.99
1497,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11106.55


In [ ]:
# Fusiona los datos de llamadas, minutos, mensajes e Internet con base en user_id y month
columns_df_data_per_user=["user_id","month","calls","message","calls_minutes","mb_used"]
df_data_per_user=pd.DataFrame(columns=columns_df_data_per_user)
month_list=['January','February','March','April','May','June','July','August','September','October','November','December']
for i in range(len(mb_used_per_user)):
    month_count=0
    while month_count < 12:
        aux_user_id=mb_used_per_user.index[i]
        try:
            aux_calls_user=calls_per_user.loc[aux_user_id,month_list[month_count]]
        except:
            aux_calls_user=0
        try:
            aux_message_user=message_per_user.loc[aux_user_id,month_list[month_count]]
        except:
            aux_message_user=0
        try:
            aux_minutes_user=minutes_per_user.loc[aux_user_id,month_list[month_count]]
        except:
            aux_minutes_user=0
        aux_mb_user=mb_used_per_user.loc[aux_user_id,month_list[month_count]]
        aux_row=[aux_user_id,month_list[month_count],aux_calls_user,aux_message_user,aux_minutes_user,aux_mb_user]
        df_data_per_user = df_data_per_user.append(pd.Series(aux_row, index=columns_df_data_per_user), ignore_index=True)
        month_count +=1
        
df_data_per_user.head(30)

In [ ]:
# Añade la información de la tarifa

column_plan=[]
column_city=[]
for i in range(len(df_data_per_user)):
    aux_user_id=df_data_per_user.iloc[i,0]
    aux_data_user=df_users[df_users["user_id"]==aux_user_id]
    aux_data_user=df_users[df_users["user_id"]==aux_user_id]
    column_plan.append(aux_data_user.iloc[0,6])
    column_city.append(aux_data_user.iloc[0,4])
df_data_per_user.info()
print("")
df_data_per_user["plan"]=column_plan
df_data_per_user["city"]=column_city
#Sabemos que usuarios pudieron no haber realizado llamadas,mensajes o uso de datos como lo vemos en el usuario 1010 y 1011
df_data_per_user.head(30)

In [ ]:
# Calcula el ingreso mensual para cada usuario
# Declaramos las variables para cada plan para realizar las operaciones matematicas conservando un mejor orden.
#PLAN SURF
surf_monthly_payment=df_plans.loc[0,"usd_monthly_pay"]
surf_monthly_minutes=df_plans.loc[0,"minutes_included"]
surf_monthly_sms=df_plans.loc[0,"messages_included"]
surf_monthly_gb=df_plans.loc[0,"mb_per_month_included"]
surf_extracost_minutes=df_plans.loc[0,"usd_per_minute"]
surf_extracost_sms=df_plans.loc[0,"usd_per_message"]
surf_extracost_gb=df_plans.loc[0,"usd_per_gb"]
#PLAN ULTIMATE
ultimate_monthly_payment=df_plans.loc[1,"usd_monthly_pay"]
ultimate_monthly_minutes=df_plans.loc[1,"minutes_included"]
ultimate_monthly_sms=df_plans.loc[1,"messages_included"]
ultimate_monthly_gb=df_plans.loc[1,"mb_per_month_included"]
ultimate_extracost_minutes=df_plans.loc[1,"usd_per_minute"]
ultimate_extracost_sms=df_plans.loc[1,"usd_per_message"]
ultimate_extracost_gb=df_plans.loc[1,"usd_per_gb"]
#Ahora creamos una lista que contendra la suma de ingresos por cada usuario
income_per_user=[]
#Comenzamos a recorrer el DataFrame
for i in range(len(df_data_per_user)):
    #Declararemos las variables que participaran en la suma para que se reseten en cada ciclo
    cost_message=0
    cost_minutes=0
    cost_gb=0
    income=0
    #Determinamos a que plan pertenece el usuario
    if df_data_per_user.iloc[i]["plan"]=="surf":
        #Condicion para realizar el calculo por exceder el limite de mensajes.
        if df_data_per_user.iloc[i]["message"] > surf_monthly_sms:
            cost_message=(df_data_per_user.iloc[i]["message"] - surf_monthly_sms)*surf_extracost_sms
            
        #Condicion para realizar el calculo por exceder el limite de minutos.  
        if df_data_per_user.iloc[i]["calls_minutes"] > surf_monthly_minutes:
            cost_minutes= (df_data_per_user.iloc[i]["calls_minutes"] - surf_monthly_minutes)*surf_extracost_minutes
            
        #Condicion para realizar el calculo por exceder el limite de GB.
        #La resta de divide por 1000 ya que los MB los convertimos GB ya que los costos se agregan por GB
        if df_data_per_user.iloc[i]["mb_used"] > surf_monthly_gb:
            cost_gb=((df_data_per_user.iloc[i]["mb_used"] - surf_monthly_gb)/1024)*surf_extracost_gb

        #Realizamos la suma de costo mensual
        income=surf_monthly_payment+cost_message+cost_minutes+cost_gb
        #Agregamos ese costo a la lista income_per_user
        income_per_user.append(income)
        
    #Realizamos lo mismo que la condicion anterior pero con el plan Ultimate
    elif df_data_per_user.iloc[i]["plan"]=="ultimate":

        #Condicion para realizar el calculo por exceder el limite de mensajes.
        if df_data_per_user.iloc[i]["message"] > ultimate_monthly_sms:
            print("entro en message")
            cost_message=(df_data_per_user[i]["message"] - ultimate_monthly_sms)*ultimate_extracost_sms
           
        #Condicion para realizar el calculo por exceder el limite de minutos.  
        if df_data_per_user.iloc[i]["calls_minutes"] > ultimate_monthly_minutes:
            cost_minutes= (df_data_per_user.iloc[i]["calls_minutes"] - ultimate_monthly_minutes)*ultimate_extracost_minutes
            
        #Condicion para realizar el calculo por exceder el limite de GB.
        #La resta de divide por 1000 ya que los MB los convertimos GB ya que los costos se agregan por GB
        if df_data_per_user.iloc[i]["mb_used"] > ultimate_monthly_gb:
            cost_gb=((df_data_per_user.iloc[i]["mb_used"] - ultimate_monthly_gb)/1024)*ultimate_extracost_gb
            
        #Realizamos la suma de costo mensual
        income=ultimate_monthly_payment+cost_message+cost_minutes+cost_gb
        #Agregamos ese costo a la lista income_per_user
        income_per_user.append(income)
    #Esta condiacion la dejamos en caso de que el dato del plan se Nulo y no entre en ninguna condicion anterior                    
    else:
        #Realizamos la suma de costo mensual
        income=0
        #Agregamos ese costo a la lista income_per_user
        income_per_user.append(income) 
    
#Fuera del ciclo FOR agregamos la lista de ingresos en el DataFrame como una nueva columna
df_data_per_user["income_per_user"]=income_per_user

#Revisamos la informacion de la tabla
df_data_per_user.info()

#Revisamos las primeras filas del DataFrame final.
df_data_per_user.head(30)

## Estudia el comportamiento de usuario

### Llamadas

In [ ]:
# Compara la duración promedio de llamadas por cada plan y por cada mes. Traza un gráfico de barras para visualizarla.
plan_user=df_users.groupby("user_id").first()
plan_user.drop(["first_name","last_name","age","city","reg_date","churn_date"],axis="columns",inplace=True)
#Creamos una tabla pivote para tener el pomedio de duracion de la llamadas por mes para cada usuario.
pivot_calls_data= df_calls.pivot_table(index='user_id',columns=df_calls["call_date"].dt.month,values='duration',aggfunc='mean')
#Reutilizamos la variable plan_user para obtener los planes de cada usuario 
calls_data=pd.concat([plan_user,pivot_calls_data],axis=1)
#El DataFrame calls_data convertimos en el index los meses y las columnas los planes que seran x , y respectivamente en el grafico.
calls_data=calls_data.pivot_table(columns=calls_data["plan"])
#Los meses estan en numeros por eso cambiaremos a su nombre por su respectivo numero
calls_data = calls_data.rename(index={1:'January',2:'February',3:'March',4:'April',5:'May',6:'June',7:'July',8:'August',9:'September',10:'October',11:'November',12:'December',})
#Graficamos.
calls_data.plot(kind="bar",figsize=(10, 10),title='Calls per plan',grid=True)
plt.show()

In [ ]:
# Compara el número de minutos mensuales que necesitan los usuarios de cada plan. Traza un histograma.
calls_data.plot(kind="hist",figsize=(10, 10),title='minutes monthly used user',grid=True)
plt.show()

In [ ]:
# Calcula la media y la varianza de la duración mensual de llamadas.
calls_data = calls_data.rename(columns={'surf':'surf_mean','ultimate':'ultimate_mean'})
#Creamos una tabla pivote para tener el pomedio de duracion de la llamadas por mes para cada usuario.
pivot_var_calls_data= df_calls.pivot_table(index='user_id',columns=df_calls["call_date"].dt.month,values='duration',aggfunc="sum")
#Reutilizamos la variable plan_user para obtener los planes de cada usuario 
var=pd.concat([plan_user,pivot_var_calls_data],axis=1)
var=var.pivot_table(columns=var["plan"])
#Los meses estan en numeros por eso cambiaremos a su nombre por su respectivo numero
surf_var=np.var(var["surf"])
ultimate_var=np.var(var["ultimate"])
surf_mean=np.mean(calls_data["surf_mean"])
ultimate_mean=np.mean(calls_data["ultimate_mean"])
#df_december["income_per_user"]=income_per_user
var_mean=pd.DataFrame(data={'surf':[surf_mean,surf_var],"ultimate":[ultimate_mean,ultimate_var]},index=["mean","var"])
print(var_mean)

In [ ]:
# Traza un diagrama de caja para visualizar la distribución de la duración mensual de llamadas
var_mean.plot(kind="box",figsize=(10, 10),title='Duration of calls: distr of surf and ultimate',grid=True)

### Mensajes

In [ ]:
# Comprara el número de mensajes que tienden a enviar cada mes los usuarios de cada plan
#Creamos una tabla pivote para tener la cantidad de mensajes por mes para cada usuario.
pivot_messages_data= df_messages.pivot_table(index='user_id',columns=df_messages["message_date"].dt.month,values="id",aggfunc='count')
#Reutilizamos la variable plan_user para obtener los planes de cada usuario 
messages_data=pd.concat([plan_user,pivot_messages_data],axis=1)
#El DataFrame calls_data convertimos en el index los meses y las columnas los planes que seran x , y respectivamente en el grafico.
messages_data=messages_data.pivot_table(columns=messages_data["plan"])
print(messages_data)
#Los meses estan en numeros por eso cambiaremos a su nombre por su respectivo numero
messages_data = messages_data.rename(index={1:'January',2:'February',3:'March',4:'April',5:'May',6:'June',7:'July',8:'August',9:'September',10:'October',11:'November',12:'December',})
#Graficamos.
messages_data.plot(kind="bar",figsize=(10, 10),title='Messages per plan',grid=True)
plt.show()

In [ ]:
# Compara la cantidad de tráfico de Internet consumido por usuarios por plan
#Creamos una tabla pivote para tener el pomedio de mb usados por mes para cada usuario.
pivot_internet_data= df_internet.pivot_table(index='user_id',columns=df_internet["session_date"].dt.month,values="mb_used",aggfunc='mean')
#Reutilizamos la variable plan_user para obtener los planes de cada usuario 
internet_data=pd.concat([plan_user,pivot_internet_data],axis=1)
#El DataFrame calls_data convertimos en el index los meses y las columnas los planes que seran x , y respectivamente en el grafico.
internet_data=internet_data.pivot_table(columns=internet_data["plan"])
print(internet_data)
#Los meses estan en numeros por eso cambiaremos a su nombre por su respectivo numero
internet_data = internet_data.rename(index={1:'January',2:'February',3:'March',4:'April',5:'May',6:'June',7:'July',8:'August',9:'September',10:'October',11:'November',12:'December',})
#Graficamos.
internet_data.plot(kind="bar",figsize=(10, 10),title='Mb used per plan',grid=True)
plt.show()

### Internet

In [ ]:
# Compara la cantidad de tráfico de Internet consumido por usuarios por plan. Traza un histograma.
internet_data.plot(kind="hist",figsize=(10, 10),title='Histogram: Mb used per plan',grid=True)
plt.show()

In [ ]:
# Calcula la media y la varianza de la duración mensual de llamadas.
internet_data = calls_data.rename(columns={'surf':'surf_mean','ultimate':'ultimate_mean'})
#Creamos una tabla pivote para tener el pomedio de duracion de la llamadas por mes para cada usuario.
pivot_var_internet_data= df_internet.pivot_table(index='user_id',columns=df_internet["session_date"].dt.month,values='mb_used',aggfunc="sum")
#Reutilizamos la variable plan_user para obtener los planes de cada usuario 
var=pd.concat([plan_user,pivot_var_calls_data],axis=1)
var=var.pivot_table(columns=var["plan"])
#Los meses estan en numeros por eso cambiaremos a su nombre por su respectivo numero
internet_surf_var=np.var(var["surf"])
internet_ultimate_var=np.var(var["ultimate"])
internet_surf_mean=np.mean(internet_data["surf_mean"])
internet_ultimate_mean=np.mean(internet_data["ultimate_mean"])

internet_var_mean=pd.DataFrame(data={'surf':[internet_surf_mean,internet_surf_var],"ultimate":[internet_ultimate_mean,internet_ultimate_var]},index=["mean","var"])
print(internet_var_mean)

In [ ]:
# Traza un diagrama de caja para visualizar la distribución mensual del trafico de internet
internet_var_mean.plot(kind="box",figsize=(10, 10),title='Mb used: distr of surf and ultimate',grid=True)

El Plan ultimate presenta un mayor trafico de internet

## Ingreso

[Del mismo modo que has estudiado el comportamiento de los usuarios, describe estadísticamente los ingresos de los planes.]

In [ ]:
# Compara la cantidad de ingresos mensuales por usuarios por plan
# Reutilizamos el DataFrame income_data_per_user para ver el comportamiento de los ingresos mensuales de usuarios por planes
#Creamos una tabla pivote para tener el pomedio de mb usados por mes para cada usuario.
income_data_per_user= df_data_per_user.pivot_table(index='month',columns="plan",values="income_per_user",aggfunc='sum')

#Graficamos.
income_data_per_user.plot(kind="bar",figsize=(10, 10),title='Incomes per plan',grid=True)
plt.show()

In [ ]:
# Compara la cantidad de tráfico de Internet consumido por usuarios por plan. Traza un histograma.
income_data_per_user.plot(kind="hist",figsize=(10, 10),title='Histogram:Incomes per plan',grid=True)
plt.show()

In [ ]:
# Calcula la media y la varianza de la duración mensual de llamadas.
income_mean_data_per_user= df_data_per_user.pivot_table(index='month',columns="plan",values="income_per_user",aggfunc='sum')
income_data_mean_per_user = calls_data.rename(columns={'surf':'surf_mean','ultimate':'ultimate_mean'})
#Creamos una tabla pivote para tener el pomedio de duracion de la llamadas por mes para cada usuario.
pivot_var_income_data= df_data_per_user.pivot_table(index='month',columns="plan",values="income_per_user",aggfunc='sum')
#Los meses estan en numeros por eso cambiaremos a su nombre por su respectivo numero
income_surf_var=np.var(pivot_var_income_data["surf"])
income_ultimate_var=np.var(pivot_var_income_data["ultimate"])
income_surf_mean=np.mean(income_data_mean_per_user["surf_mean"])
income_ultimate_mean=np.mean(income_data_mean_per_user["ultimate_mean"])
#df_december["income_per_user"]=income_per_user
income_var_mean=pd.DataFrame(data={'surf':[income_surf_mean,income_surf_var],"ultimate":[income_ultimate_mean,income_ultimate_var]},index=["mean","var"])
print(income_var_mean)
# Traza un diagrama de caja para visualizar la distribución de la duración mensual de llamadas
income_var_mean.plot(kind="box",figsize=(10, 10),title='Incomes per plan: distr of surf and ultimate',grid=True)

Podemos observar que en comparacion el plan que produce una mayor cantidad de ingresos es el plan User

## Prueba las hipótesis estadísticas

In [ ]:
# Prueba las hipótesis

alpha=0.005
income_test=df_data_per_user.pivot_table(index="user_id",columns="plan",values="income_per_user",aggfunc="mean")
income_test=income_test.fillna(0)
income_surf=income_test["surf"]
income_ultimate=income_test["ultimate"]
print(income_surf)
print(income_ultimate)
# realizar una prueba
results = st.ttest_ind(income_surf,income_ultimate,equal_var = False)
# imprimir el valor p resultante
print('valor p: ', results.pvalue)

# comparar el valor p con el umbral

if (results.pvalue < alpha):
    print("Rechazamos la hipótesis nula")
else:
    print("No podemos rechazar la hipótesis nula") 

In [ ]:
# Prueba las hipótesis
#New York-Newark-Jersey City, NY-NJ-PA MSA
# Prueba las hipótesis

alpha=0.005
income_test=df_data_per_user.pivot_table(index="user_id",columns="city",values="income_per_user",aggfunc="mean")
income_test=income_test.fillna(0)
income_NY_NJ=income_test["New York-Newark-Jersey City, NY-NJ-PA MSA"]

#Borramos la columna que no queremos que sea co
income_test.drop(['New York-Newark-Jersey City, NY-NJ-PA MSA'], axis=1)
income_other_cities=income_test
# realizar una prueba
results = st.ttest_ind(income_surf,income_ultimate,equal_var = False)
# imprimir el valor p resultante
print('valor p: ', results.pvalue)

# comparar el valor p con el umbral

if (results.pvalue < alpha):
    print("Rechazamos la hipótesis nula")
else:
    print("No podemos rechazar la hipótesis nula") 

## Conclusión general

Podemos determinar que la utilidad que los usuarios dan sus planes son muy similares segun los graficos de consumo de internet, llamadas y mensajes, las diferencias son demasiados leves en las comparaciones, pero cuando hablamos de ingresos, podemos observar que el plan Surf supera por mucho al plan Ultimate, esto se debe a:

*La cantidad de usuarios en el plan surf supera a la cantidad de usuarios del plan ultimate.

*El plan surf al tener menos llamandas, mensajes y mb disponibles en comparacion al plan ultimate los usuarios tienden a sobrepasar los limites de este teniendo que pagar las tarifas extras del plan surf.